# Step-by-Step documentation
## Part I: Time, TimeSpan, Event

This notebook is meant for anyone to better understand the underlying structures which are used in py-vortex, and to a larger extent in Tidal. <br>
Beware that there may be some differences, but at least, this aims to cover the current `py-vortex`implementation in Python hoping to explain it a bit more at low-level<br> 
There are a couple crucial class in py-vortex, namely Time, TimeSpan, and Event, we will go through in details about how they work.<br>
Part II will focus on Pattern only

In [1]:
from py_vortex import *

# Time
Time class is a subclass of fraction. While instanciating a Time object, you can specify it by either one or two floats.<br>
If you do so with two numbers, then the fraction is simply the ratio of them<br>
If you create it with only one number, it translates it into a fraction<br>
Apparently, irrationals does work too (probably rounded)

In [2]:
Time(1,2)

Time(1, 2)

In [3]:
Time(0.5)

Time(1, 2)

In [4]:
Time(0.12342134234)

Time(8893444981951847, 72057594037927936)

In [5]:
from math import pi
Time(pi)

Time(884279719003555, 281474976710656)

## Methods

Time Class has currently 3 methods, `sam`, `next_sam` and `whole_cycle`. <br>
Sam refers to indian music, where *sam* is the denomination of the first beat of the cycle.<br>
Trivially, `next_sam` is the first beat of the next cycle<br>
And whole_duration is a TimeSpan (see below) of a whole cycle <br>

### sam

In [6]:
Time(0.43).sam()

Time(0, 1)

### next_sam

In [7]:
Time(0.43).next_sam()

Fraction(1, 1)

### whole_cycle

In [8]:
Time(0.42).whole_cycle()

TimeSpan(Time(0, 1), Time(1, 1))

In [9]:
Time(-2.3).whole_cycle()

TimeSpan(Time(-3, 1), Time(-2, 1))

# TimeSpan
As introduced right above, TimeSpan represents a structure that has a certain duration<br>
It is commonly used in Event, as will be talked about later.
It has 2 attributes: `begin` and `end` and 3 methods `spanCycles`, `withTime` and `sect` <br>
We will not count `maybeSect` here since it is just a method for dealing with sect corner cases.<br>
TimeSpan `begin` and `end` attributes are themselves `Time` objects. <br>
In theory, nothing restricts them to be such like $begin > end$ but in practive it is not the case.<br>

In [10]:
TimeSpan(-0.1, 0.8)

TimeSpan(Time(-3602879701896397, 36028797018963968), Time(3602879701896397, 4503599627370496))

In [11]:
TimeSpan(1/8, 1/4)

TimeSpan(Time(1, 8), Time(1, 4))

## Methods

TimeSpan has some critical methods relevant to how Patterns are working, and are used extensively in py-vortex.<br>
`spanCycles` allows for a TimeSpan to extend its values to its nearest sam and next_sam.<br>
`withtime` allows to apply a function to the begining and the end values of a TimeSpan, basically it allows for a Pattern to modify the duration of an Event (for example) or to the moment it is queried.<br>
`sect` is also really useful when querying a pattern, to determine the duration of event within a given cycle.<br>
These are just examples of how they can be used, to have a better understanding but they are no such strict rules and everything is possible.

### span_cycles

In [12]:
TimeSpan(0,1).span_cycles()

[TimeSpan(Time(0, 1), Time(1, 1))]

In [13]:
TimeSpan(-0.25, 0.75).span_cycles()

[TimeSpan(Time(-1, 4), Time(0, 1)), TimeSpan(Time(0, 1), Time(3, 4))]

In [14]:
TimeSpan(-2.25, 3.45).span_cycles()

[TimeSpan(Time(-9, 4), Time(-2, 1)),
 TimeSpan(Time(-2, 1), Time(-1, 1)),
 TimeSpan(Time(-1, 1), Time(0, 1)),
 TimeSpan(Time(0, 1), Time(1, 1)),
 TimeSpan(Time(1, 1), Time(2, 1)),
 TimeSpan(Time(2, 1), Time(3, 1)),
 TimeSpan(Time(3, 1), Time(3884354678607053, 1125899906842624))]

### with_time

In [15]:
TimeSpan(0.25, 1.25).with_time(lambda x: x*2)

TimeSpan(Time(1, 2), Time(5, 2))

Ok, let's stop a bit here and understand what's happening.<br>
We are defining a lambda function that apply the operation multiply by 2 on each element of the TimeSpan, such that 0.25 and 1.25 become 0.5 and 2.5<br>
If you are not comfortable with `lambda` syntax, you can simply declare you function the standard way and pass it to `withTime` in such way:

In [16]:
def multiply_by_2(x):
    return x * 2

TimeSpan(0.25, 1.25).with_time(multiply_by_2)

TimeSpan(Time(1, 2), Time(5, 2))

And the result is exactly the same.

### sect

If you need to know what is the TimeSpan common to 2 different TimeSpans, you can just use `sect` 

In [17]:
TimeSpan(0, 0.75).sect(TimeSpan(0.5, 1.5))

TimeSpan(Time(1, 2), Time(3, 4))

In [18]:
TimeSpan(0.5, 1.75).sect(TimeSpan(0, 0.75))

TimeSpan(Time(1, 2), Time(3, 4))

If they do not overlap, then the result is None

In [19]:
no_overlap = TimeSpan(0, 0.75).sect(TimeSpan(1.5, 2))
no_overlap is None

True

# Event

Everythin in TIdal is an event, and Pattern (don't worry will get to it) modifies them.<br>
Then, the scheduler who is responsible to making happen event with regards to Time, queries then and make them alive,<br>
An event is composed of three different constituents: 
- a whole (a TimeSpan instance) which represents the duration an event is active, regardless of the query 
- a part (a TimeSpan instance) which represents the duration an event is active, during the TimeSpan of the cycle it is queried
- a value (a string, a number, etc... ) which identifies it. Think of it as a key.

Event possesses methods `withSpan`, `withValue` and `hasOnset`. These methods modifies with functions the differents constituents of an Event

In [20]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello")

Event(TimeSpan(Time(1, 4), Time(5, 4)), TimeSpan(Time(1, 2), Time(3, 4)), 'hello')

In [21]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), 2)

Event(TimeSpan(Time(1, 4), Time(5, 4)), TimeSpan(Time(1, 2), Time(3, 4)), 2)

### with_span

In [22]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").with_span(lambda span: span.with_time(lambda x: x+2))

Event(TimeSpan(Time(9, 4), Time(13, 4)), TimeSpan(Time(5, 2), Time(11, 4)), 'hello')

Ok, just a moment of relfexion here. Even though `withSpan` takes a function as a parameter, it cannot take *any* function.<br>
For this to be working, this function has to be a method *known* to TimeSpan (i.e. spanCycles, withTime or sect).<br>
For example, this does not work (unless the + operator is overloaded for class TimeSpan - which is not the case atm).

In [23]:
# This does not work !!!
# Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").withSpan(lambda x: x+2)

Also, be aware that any var pass to lambda will work, we use `span` here for convenience but it could be *anything*<br>

In [24]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").with_span(lambda x: x.with_time(lambda x: x+2))

Event(TimeSpan(Time(9, 4), Time(13, 4)), TimeSpan(Time(5, 2), Time(11, 4)), 'hello')

In [25]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").with_span(lambda span: span.span_cycles())

Event([TimeSpan(Time(1, 4), Time(1, 1)), TimeSpan(Time(1, 1), Time(5, 4))], [TimeSpan(Time(1, 2), Time(3, 4))], 'hello')

In [26]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").with_span(lambda span: span.sect(TimeSpan(0,2)))

Event(TimeSpan(Time(1, 4), Time(5, 4)), TimeSpan(Time(1, 2), Time(3, 4)), 'hello')

### with_value

In [27]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").with_value(lambda x: x.upper())

Event(TimeSpan(Time(1, 4), Time(5, 4)), TimeSpan(Time(1, 2), Time(3, 4)), 'HELLO')

In [28]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), 10).with_value(lambda x: x + 2)

Event(TimeSpan(Time(1, 4), Time(5, 4)), TimeSpan(Time(1, 2), Time(3, 4)), 12)

### has_onset

In [29]:
Event(TimeSpan(0.25, 1.25), TimeSpan(0.5, 0.75), "hello").has_onset()

False

In [30]:
Event(TimeSpan(0.5, 1.25), TimeSpan(0.5, 0.75), "hello").has_onset()

True